In [2]:
import geopandas as gpd
from shapely.geometry import Point
from google.cloud import bigquery
import pandas as pd
import datetime as dt

# Set up client
client = bigquery.Client(project="drexel-msds")


c:\Users\iauge\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
# Define target years and columns
years = range(2010, 2021)

columns = [
    "geo_id", "median_income", "total_pop", "median_age",
    "white_pop", "black_pop", "hispanic_pop", "asian_pop",
    "bachelors_degree_or_higher_25_64",
    "less_than_high_school_graduate",
    "some_college_and_associates_degree",
    "different_house_year_ago_same_city",
    "different_house_year_ago_different_city",
    "median_rent",
    "percent_income_spent_on_rent",
    "rent_over_50_percent",
    "poverty", "gini_index"
]

# Dynamically generate SQL with UNION ALL
selects = []
for year in years:
    selects.append(f"""
    SELECT 
        {', '.join(columns)},
        {year} AS year
    FROM `bigquery-public-data.census_bureau_acs.county_{year}_5yr`
    """)

query = "\nUNION ALL\n".join(selects)

# Run the query
df = client.query(query).to_dataframe()

# Force geo_id to string with zero-padding if needed
df["geo_id"] = df["geo_id"].astype(str).str.zfill(5)

# Extract state and county FIPS codes
df["state_fips"] = df["geo_id"].str[:2]
df["county_fips"] = df["geo_id"].str[2:]

# Reorder columns 
cols = ["geo_id", "state_fips", "county_fips", "year"] + [
    col for col in df.columns if col not in ["geo_id", "state_fips", "county_fips", "year"]
]
df = df[cols]

# Save locally
df.to_csv("acs_multi_year_panel.csv", index=False)

print("Data downloaded and saved as 'acs_multi_year_panel.csv'")


In [ ]:
# Target years
years = list(range(2000, 2025))

# Core metrics: mean temp, max temp, precipitation
base_columns = [
    "stn", "wban", "year", "mo", "da", "temp", "max", "prcp",
    "hail", "thunder", "tornado_funnel_cloud"
]

# Prepare query for each year
# Where statement addresses null values
queries = []
for year in years:
    table = f"`bigquery-public-data.noaa_gsod.gsod{year}`"
    queries.append(f"""
        SELECT 
            {', '.join(base_columns)}
        FROM {table}
        WHERE temp < 9999.9
          AND prcp < 99.99
          AND max < 9999.9
    """)

# Combine into one big UNION ALL query
union_query = "\nUNION ALL\n".join(queries)

# Run and load into DataFrame
df = client.query(union_query).to_dataframe()

# Create full date field
df["date"] = pd.to_datetime(df[["year", "mo", "da"]].astype(str).agg("-".join, axis=1), errors="coerce")

# Preview + save
print(df.head())
df.to_csv("gsod_daily_2000_2024.csv", index=False)

print("GSOD daily data downloaded and saved.")

In [ ]:
def process_gsod_by_year(filepath, output_file, years=range(2000, 2025), chunksize=1_000_000):
    gsod_dtypes = {
        "stn": str,
        "wban": str,
        "year": int,
        "mo": int,
        "da": int,
        "temp": float,
        "max": float,
        "prcp": float,
        "hail": str,
        "thunder": str,
        "tornado_funnel_cloud": str
    }

    all_results = []

    for year in years:
        print(f"Processing year: {year}")
        year_chunks = []

        chunk_iter = pd.read_csv(
            filepath,
            dtype=gsod_dtypes,
            parse_dates=["date"],
            chunksize=chunksize,
            low_memory=False
        )

        # Collect chunks for the given year
        for chunk in chunk_iter:
            chunk = chunk[chunk["year"] == year]
            if not chunk.empty:
                year_chunks.append(chunk)

        if not year_chunks:
            print(f"No data for {year}")
            continue

        year_df = pd.concat(year_chunks, ignore_index=True)

        # Add derived fields
        year_df["quarter"] = year_df["date"].dt.quarter
        year_df["heat_day_90f"] = year_df["max"] > 90.0
        year_df["stn"] = year_df["stn"].astype(str).str.zfill(6)
        year_df["wban"] = year_df["wban"].astype(str).str.zfill(5)
        year_df["station_id"] = year_df["stn"] + "-" + year_df["wban"]

        # Convert flags to binary
        for col in ["hail", "thunder", "tornado_funnel_cloud"]:
            year_df[col] = pd.to_numeric(year_df[col], errors="coerce").fillna(0).astype("int8")

        # Group and aggregate
        grouped = year_df.groupby(["station_id", "year", "quarter"]).agg(
            avg_temp=("temp", "mean"),
            max_temp=("max", "max"),
            total_precip=("prcp", "sum"),
            heat_days_90F=("heat_day_90f", "sum"),
            hail_days=("hail", "sum"),
            thunder_days=("thunder", "sum"),
            tornado_days=("tornado_funnel_cloud", "sum"),
            num_days=("date", "count")
        ).reset_index()

        all_results.append(grouped)
        print(f"Year {year} processed: {len(grouped)} station-quarter records.")

    # Final output
    full_df = pd.concat(all_results, ignore_index=True)
    full_df.to_csv(output_file, index=False)
    print(f"\nAll years saved to: {output_file}")


In [ ]:
process_gsod_by_year(
    filepath="gsod_daily_2000_2024.csv",
    output_file="gsod_quarterly_aggregates.csv"
)


In [3]:
years = range(2000, 2025) 
all_dfs = []

for year in years:
    print(f"Querying storm events for {year}...")

    query = f"""
    SELECT
      CONCAT(LPAD(state_fips_code, 2, '0'), LPAD(cz_fips_code, 3, '0')) AS geo_id,
      EXTRACT(YEAR FROM event_begin_time) AS year,
      event_type,
      COUNT(*) AS num_events,
      SUM(damage_property) AS total_property_damage,
      SUM(damage_crops) AS total_crop_damage,
      SUM(injuries_direct + injuries_indirect) AS total_injuries,
      SUM(deaths_direct + deaths_indirect) AS total_deaths
    FROM
      `bigquery-public-data.noaa_historic_severe_storms.storms_{year}`
    WHERE
      cz_type = 'C'
    GROUP BY
      geo_id, year, event_type
    """

    df = client.query(query).to_dataframe()
    all_dfs.append(df)

# Combine all years
storm_df = pd.concat(all_dfs, ignore_index=True)
print("All storm event data loaded:", storm_df.shape)

# Optional: Save to CSV
storm_df.to_csv("storm_events_by_county_year.csv", index=False)


Querying storm events for 2000...
Querying storm events for 2001...
Querying storm events for 2002...
Querying storm events for 2003...
Querying storm events for 2004...
Querying storm events for 2005...
Querying storm events for 2006...
Querying storm events for 2007...
Querying storm events for 2008...
Querying storm events for 2009...
Querying storm events for 2010...
Querying storm events for 2011...
Querying storm events for 2012...
Querying storm events for 2013...
Querying storm events for 2014...
Querying storm events for 2015...
Querying storm events for 2016...
Querying storm events for 2017...
Querying storm events for 2018...
Querying storm events for 2019...
Querying storm events for 2020...
Querying storm events for 2021...
Querying storm events for 2022...
Querying storm events for 2023...
Querying storm events for 2024...
All storm event data loaded: (215730, 8)


In [ ]:
storm_wide = pivot_storm_events_by_type(storm_df)
print(storm_wide.dtypes)

In [ ]:
counties = gpd.read_file(r"cb_2023_us_county_5m\cb_2023_us_county_5m.shp")

# Project to WGS84 (matches lat/lon)
counties = counties.to_crs("EPSG:4326")

# Create a unique FIPS code for join
counties["geo_id"] = counties["STATEFP"] + counties["COUNTYFP"]

# Select relevant columns
counties = counties[["geo_id", "NAME", "STATEFP", "COUNTYFP", "geometry"]]
counties.head()


In [ ]:
query = """
SELECT 
  usaf,
  wban,
  name,
  country,
  state,
  lat,
  lon,
  elev,
  `begin`,
  `end`
FROM 
  `bigquery-public-data.noaa_gsod.stations`
"""

stations_df = client.query(query).to_dataframe()

stations_df["usaf"] = stations_df["usaf"].str.zfill(6)
stations_df["wban"] = stations_df["wban"].str.zfill(5)
stations_df["station_id"] = stations_df["usaf"] + "-" + stations_df["wban"]
stations_df["begin"] = pd.to_datetime(stations_df["begin"], format="%Y%m%d", errors="coerce")
stations_df["end"] = pd.to_datetime(stations_df["end"], format="%Y%m%d", errors="coerce")

stations_df = stations_df[stations_df["country"].str.upper() == "US"]

# Filter for stations active at any point during analysis window
mask = (stations_df["begin"] <= "2024-12-31") & (stations_df["end"] >= "2000-01-01")
stations_df = stations_df[mask]

stations_df.to_csv("noaa_stations.csv", index=False)
stations_df.head()
